# Recipe 13: Compare Data Coverage Across Multiple Targets

This notebook demonstrates how to analyze and compare how much data is available for different planetary bodies in the PDS.

## What You'll Learn
- How to programmatically compare data availability across targets
- Working with Context to find multiple targets
- Aggregating query results for analysis
- Creating comparison DataFrames

In [ ]:
import pds.peppi as pep
import pandas as pd

## Setup

First, create a client connection and context object:

In [ ]:
client = pep.PDSRegistryClient()
context = pep.Context()

## Define Targets to Compare

Let's compare data coverage across the terrestrial planets:

In [ ]:
target_names = ["Mars", "Jupiter", "Saturn", "Venus", "Mercury"]

## Query Each Target

For each target, we'll query for observational products and collect statistics:

In [ ]:
results = []

for target_name in target_names:
    print(f"Querying {target_name}...")
    
    # Get target info
    targets = context.TARGETS.search(target_name)
    if not targets:
        print(f"  No target found for {target_name}")
        continue
    
    target = targets[0]
    print(f"  Found: {target.name} ({target.type})")
    
    # Count products (using a sample)
    products = pep.Products(client) \
        .has_target(target.lid) \
        .observationals()
    
    df = products.as_dataframe(max_rows=100)
    
    sample_count = len(df) if df is not None else 0
    print(f"  Sample count: {sample_count}")
    
    results.append({
        'Target': target.name,
        'Type': target.type,
        'LID': target.lid,
        'Sample Count (first 100)': sample_count
    })

## Create Comparison DataFrame

In [ ]:
comparison_df = pd.DataFrame(results)
comparison_df

## Visualize the Comparison

Let's create a simple bar chart to visualize the data coverage:

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.bar(comparison_df['Target'], comparison_df['Sample Count (first 100)'])
plt.xlabel('Target')
plt.ylabel('Sample Product Count')
plt.title('PDS Data Coverage by Target (Sample of First 100 Products)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## Key Takeaways

- Context search makes it easy to find targets by name
- You can iterate over multiple targets programmatically
- DataFrames and visualization help compare coverage
- Sampling (max_rows) keeps queries fast for exploratory analysis

## Next Steps

- Try comparing different categories (asteroids vs planets)
- Add more detailed statistics (by instrument, processing level)
- Remove max_rows limit for actual counts (will take longer)
- Create time-series comparisons